In [2]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpfwlmhr31".


In [30]:
%%cuda

// Include necessary standard libraries
#include <stdio.h>
#include <cmath>
#include <iostream>
#include <chrono>

using namespace std;
using namespace std::chrono;

// Source: adapted from https://onezork.wordpress.com/2014/08/29/gpu-mergesort/
__host__
void mergesort(long*, long, dim3, dim3);
__global__
void gpu_mergesort(long*, long*, long, long, long, dim3*, dim3*);

// Device function, calculate Euclidean distance between two points
__device__
double calculateDistance(const float2 p1, const float2 p2) {
    return sqrt((p1.x - p2.x) * (p1.x - p2.x) + (p1.y - p2.y) * (p1.y - p2.y));
}

// CUDA kernel, perform merge sort on arrays of distances
__global__
void gpu_mergesort(float* source, float* dest, long size, long width, long slices, dim3* threads, dim3* blocks) {
    int x;
    // Calculate "global" index of thread
    unsigned int idx = threadIdx.x + threadIdx.y * (x  = threads->x) + threadIdx.z * (x *= threads->y) +
           blockIdx.x  * (x *= threads->z) + blockIdx.y  * (x *= blocks->z) + blockIdx.z  * (x *= blocks->y);
    long start = width*idx*slices,
         middle,
         end;

    // Perform merge sort on slices of the array
    for (long slice = 0; slice < slices; slice++) {
        if (start >= size)
            break;

        middle = min(start + (width >> 1), size);
        end = min(start + width, size);

        long i = start;
        long j = middle;
        for (long k = start; k < end; k++) {
            if (i < middle && (j >= end || source[i] < source[j])) {
                dest[k] = source[i];
                i++;
            } else {
                dest[k] = source[j];
                j++;
            }
        }
        start += width;
    }
    __syncthreads();
}

// CUDA kernel, compute distances to a query point
__global__
void qnear_kernel(float2* d_points, float* d_dist, int* d_idx, float2 qrand, int n) {
    for (int idx = threadIdx.x; idx < n; idx += blockDim.x) {
        d_dist[idx] = calculateDistance(d_points[idx], qrand);
    }
    __syncthreads();
}

// Host function, sort the neighbors by increasing distance from query point
__host__
int qnear(float2* h_points, float2 qrand, int n, bool verbose=false) {
    // Set up device and host memory
    float2 *d_points;
    int *d_idx;
    float *d_dest;
    float *d_dist;
    float h_dist[n];
    cudaMalloc(&d_points, n * sizeof(float2));
    cudaMalloc(&d_idx, n * sizeof(int));
    cudaMalloc(&d_dist, n * sizeof(float));
    cudaMalloc(&d_dest, n * sizeof(float));
    cudaMemcpy(d_points, h_points, n * sizeof(float2), cudaMemcpyHostToDevice);
    
    // Set up threads and blocks for CUDA kernel
    dim3* D_threads;
    dim3* D_blocks;
    long nThreads = 8; // Note: number of threads must be divisible by number of points
    dim3 threadsPerBlock(nThreads, 1, 1);
    dim3 blocksPerGrid(1, 1, 1);
    cudaMalloc((void**) &D_threads, sizeof(dim3));
    cudaMalloc((void**) &D_blocks, sizeof(dim3));
    cudaMemcpy(D_threads, &threadsPerBlock, sizeof(dim3), cudaMemcpyHostToDevice);
    cudaMemcpy(D_blocks, &blocksPerGrid, sizeof(dim3), cudaMemcpyHostToDevice);
    
    // Calculate distances to query point
    qnear_kernel<<<blocksPerGrid, threadsPerBlock, n*sizeof(float)>>>(d_points, d_dist, d_idx, qrand, n);
    cudaDeviceSynchronize();
    cudaMemcpy(h_dist, d_dist, n * sizeof(float), cudaMemcpyDeviceToHost);
    
    if (verbose) {
        cout << "-----BEFORE-----" << endl;
        for (int i = 0; i < n; i++) {
            cout << i << ": " << h_dist[i] << endl;
        }
    }
    
    float* A = d_dist;
    float* B = d_dest;
    
    // Perform mergesort
    long size = n;
    for (int width = 2; width < (size << 1); width <<= 1) {
        long slices = size / ((nThreads) * width) + 1;
        gpu_mergesort<<<blocksPerGrid, threadsPerBlock>>>(A, B, size, width, slices, D_threads, D_blocks);

        // Swap the input / output arrays
        A = A == d_dist ? d_dest : d_dist;
        B = B == d_dist ? d_dest : d_dist;
    }    
    cudaMemcpy(h_dist, A, size * sizeof(float), cudaMemcpyDeviceToHost);
    
    if (verbose) {
        cout << "-----AFTER-----" << endl;
        for (int i = 0; i < n; i++) {
            cout << i << ": " << h_dist[i] << endl;
        }
    }
    
    // Free CUDA memory
    cudaFree(A);
    cudaFree(B);
    cudaFree(d_idx);
    cudaFree(d_points);
    
    return 0;
}

__host__
int main() {
    // Create n points uniformly in [100, 100] grid
    int n = 16;
    float2 h_points[n];
    float side = sqrt(n);  // Number of points along one side of the grid
    float step_x = 100.0 / (side - 1);  // Step size for x coordinate
    float step_y = 100.0 / (side - 1);  // Step size for y coordinate
    for (int i = 0; i < side; i++) {
        for (int j = 0; j < side; j++) {
            int index = i * side + j;
            h_points[index].x = j * step_x;
            h_points[index].y = i * step_y;
        }
    }
    float2 qrand = {5, 5}; // Query point

    
    
    // Uncomment out to run k-Nearest Neighbors once in Verbose mode
    cout << "-----POINTS-----" << endl;
    for (int i = 0; i < n; i++) {
        cout << "[" << h_points[i].x << ", " << h_points[i].y << "]" << endl;
    }
    
    auto start_time = high_resolution_clock::now();
    
    int nearestIdx = qnear(h_points, qrand, n, true); // Verbose mode is true
    
    // Note: execution times longer if distances are outputted
    auto end_time = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(end_time - start_time);
    cout << "Execution time: " << duration.count() << " microseconds" << endl;    

    
    
    // Uncomment out to get execution time of n trials
    /*
    microseconds totalDuration(0);
    
    int n_trials = 100;
    cout << "Computing k-Nearest Neighbors: " << n_trials << " times" << endl;

    for (int i = 0; i < n_trials; ++i) {
        auto start_time = high_resolution_clock::now();

        // Call the nearest neighbor function
        int nearestIdx = qnear(h_points, qrand, n);

        auto end_time = high_resolution_clock::now();
        auto duration = duration_cast<microseconds>(end_time - start_time);
        totalDuration += duration;
        
        cout << ".";
    }
    cout << endl;

    // Calculate average execution time
    auto averageDuration = totalDuration.count() / n_trials;
    cout << "Average execution time: " << averageDuration << " microseconds" << endl;
    */
    
    return 0;
}

-----POINTS-----
[0, 0]
[33.3333, 0]
[66.6667, 0]
[100, 0]
[0, 33.3333]
[33.3333, 33.3333]
[66.6667, 33.3333]
[100, 33.3333]
[0, 66.6667]
[33.3333, 66.6667]
[66.6667, 66.6667]
[100, 66.6667]
[0, 100]
[33.3333, 100]
[66.6667, 100]
[100, 100]
-----BEFORE-----
0: 7.07107
1: 28.7711
2: 61.869
3: 95.1315
4: 28.7711
5: 40.0694
6: 67.8642
7: 99.1351
8: 61.869
9: 67.8642
10: 87.2098
11: 113.26
12: 95.1315
13: 99.1351
14: 113.26
15: 134.35
-----AFTER-----
0: 7.07107
1: 28.7711
2: 28.7711
3: 40.0694
4: 61.869
5: 61.869
6: 67.8642
7: 67.8642
8: 87.2098
9: 95.1315
10: 95.1315
11: 99.1351
12: 99.1351
13: 113.26
14: 113.26
15: 134.35
Execution time: 712318 microseconds

